# 06_Interactive_Dashboard

## Summary Dashboard & Basic Analytics

This notebook provides:
- Basic summary metrics
- Simple SQL queries for insights
- Basic visualizations

---
## Configuration

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
import matplotlib.pyplot as plt

spark = SparkSession.getActiveSession()

# Table references
GOLD_TABLE = "cost_aware_capstone.risk_decisioning.gold_decision_recommendations"
SILVER_TABLE = "cost_aware_capstone.risk_decisioning.silver_cost_aware_features"

print("Configuration loaded")

---
## 1. Basic Summary Statistics

In [ ]:
# Query 1: Count total cases and investigated cases
summary = spark.sql(f"""
    SELECT
        COUNT(*) AS total_cases,
        SUM(decision) AS investigated,
        ROUND(AVG(risk_probability), 3) AS avg_risk
    FROM {GOLD_TABLE}
""")

display(summary)

In [ ]:
# Query 2: Total expected savings
savings = spark.sql(f"""
    SELECT
        ROUND(SUM(expected_savings_if_investigated), 2) AS total_potential_savings,
        ROUND(SUM(CASE WHEN decision = 1 THEN expected_savings_if_investigated ELSE 0 END), 2) AS captured_savings
    FROM {GOLD_TABLE}
""")

display(savings)

---
## 2. Decision Breakdown

In [ ]:
# Query 3: Decision breakdown
decision_breakdown = spark.sql(f"""
    SELECT
        CASE WHEN decision = 1 THEN 'Investigated' ELSE 'Not Investigated' END AS status,
        COUNT(*) AS count,
        ROUND(AVG(risk_probability), 3) AS avg_risk,
        ROUND(AVG(expected_savings_if_investigated), 2) AS avg_savings
    FROM {GOLD_TABLE}
    GROUP BY decision
    ORDER BY decision DESC
""")

display(decision_breakdown)

In [ ]:
# Simple bar chart: Investigated vs Not Investigated
df = decision_breakdown.toPandas()

plt.figure(figsize=(8, 5))
plt.bar(df['status'], df['count'], color=['green', 'gray'])
plt.xlabel('Decision')
plt.ylabel('Number of Cases')
plt.title('Cases: Investigated vs Not Investigated')
plt.show()

---
## 3. Top 10 High-Value Cases

In [ ]:
# Query 4: Top 10 cases by expected savings
top_cases = spark.sql(f"""
    SELECT
        case_id,
        ROUND(risk_probability, 3) AS risk,
        ROUND(expected_savings_if_investigated, 2) AS expected_savings,
        priority_rank
    FROM {GOLD_TABLE}
    ORDER BY expected_savings_if_investigated DESC
    LIMIT 10
""")

display(top_cases)

---
## 4. Risk Distribution

In [ ]:
# Query 5: Risk distribution buckets
risk_buckets = spark.sql(f"""
    SELECT
        CASE
            WHEN risk_probability < 0.2 THEN 'Low (0-20%)'
            WHEN risk_probability < 0.5 THEN 'Medium (20-50%)'
            WHEN risk_probability < 0.8 THEN 'High (50-80%)'
            ELSE 'Very High (80%+)'
        END AS risk_level,
        COUNT(*) AS case_count,
        SUM(decision) AS investigated
    FROM {GOLD_TABLE}
    GROUP BY
        CASE
            WHEN risk_probability < 0.2 THEN 'Low (0-20%)'
            WHEN risk_probability < 0.5 THEN 'Medium (20-50%)'
            WHEN risk_probability < 0.8 THEN 'High (50-80%)'
            ELSE 'Very High (80%+)'
        END
    ORDER BY MIN(risk_probability)
""")

display(risk_buckets)

In [ ]:
# Bar chart: Risk distribution
risk_df = risk_buckets.toPandas()

plt.figure(figsize=(10, 5))
plt.bar(risk_df['risk_level'], risk_df['case_count'], color='steelblue')
plt.xlabel('Risk Level')
plt.ylabel('Number of Cases')
plt.title('Case Distribution by Risk Level')
plt.xticks(rotation=15)
plt.show()

---
## 5. Strategy Comparison

In [ ]:
# Query 6: Compare our approach vs risk-first baseline
comparison = spark.sql(f"""
    WITH ranked AS (
        SELECT
            expected_savings_if_investigated,
            ROW_NUMBER() OVER (ORDER BY expected_savings_if_investigated DESC) AS cost_aware_rank,
            ROW_NUMBER() OVER (ORDER BY risk_probability DESC) AS risk_first_rank
        FROM {GOLD_TABLE}
    )
    SELECT
        'Cost-Aware (Ours)' AS strategy,
        ROUND(SUM(CASE WHEN cost_aware_rank <= 50 THEN expected_savings_if_investigated END), 2) AS savings
    FROM ranked
    UNION ALL
    SELECT
        'Risk-First (Baseline)',
        ROUND(SUM(CASE WHEN risk_first_rank <= 50 THEN expected_savings_if_investigated END), 2)
    FROM ranked
""")

display(comparison)

In [ ]:
# Bar chart: Strategy comparison
comp_df = comparison.toPandas()

plt.figure(figsize=(8, 5))
colors = ['green', 'orange']
plt.barh(comp_df['strategy'], comp_df['savings'], color=colors)
plt.xlabel('Expected Savings ($)')
plt.title('Strategy Comparison (50 Investigations)')
plt.show()

---
## 6. Final Summary

In [ ]:
# Get key metrics
summary_df = spark.sql(f"""
    SELECT
        COUNT(*) AS total_cases,
        SUM(decision) AS investigated,
        ROUND(SUM(CASE WHEN decision = 1 THEN expected_savings_if_investigated END), 2) AS total_savings
    FROM {GOLD_TABLE}
""").toPandas()

print("="*50)
print("       COST-AWARE AI DECISION SYSTEM SUMMARY")
print("="*50)
print(f"Total Cases:        {summary_df['total_cases'].values[0]:,}")
print(f"Cases Investigated: {summary_df['investigated'].values[0]:,.0f}")
print(f"Expected Savings:   ${summary_df['total_savings'].values[0]:,.2f}")
print("="*50)

---
## Summary

This dashboard showed:
1. Basic volume and savings metrics
2. Decision breakdown (investigated vs not)
3. Top high-value cases
4. Risk distribution across cases
5. Strategy comparison proving cost-aware approach works better

**Key Finding**: Cost-aware prioritization captures more savings than traditional risk-first approach.

# 06_Interactive_Dashboard

## Executive Summary & Visual Analytics

This notebook provides:
- **Executive Summary** with key business metrics
- **SQL-based analysis** for deep insights
- **Interactive visualizations** for stakeholder presentations
- **ROC curves** and model performance visuals
- **Cost-benefit analysis** charts

**Target Audience**: Business stakeholders, risk managers, executives

In [ ]:
# ============================================
# 06_Interactive_Dashboard.ipynb
# --------------------------------------------
# Purpose:
#   Create executive-ready visualizations
#   and interactive analytics dashboards
#   for the Cost-Aware Decision System.
#
# Key Outputs:
#   - Executive summary metrics
#   - ROC and performance curves
#   - Cost-benefit analysis charts
#   - Capacity sensitivity visuals
#
# Evaluation Alignment:
#   - Business Impact & Practical Use
#   - Documentation & Explainability
#   - AI Innovation & Insight Generation
# ============================================

---
## 1. Configuration & Imports

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, sum as spark_sum, avg, count, when,
    round as spark_round, lit, percentile_approx
)
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Initialize Spark
spark = SparkSession.getActiveSession()

# Table references
CATALOG = "cost_aware_capstone"
SCHEMA = "risk_decisioning"

GOLD_TABLE = f"{CATALOG}.{SCHEMA}.gold_decision_recommendations"
RISK_TABLE = f"{CATALOG}.{SCHEMA}.ml_risk_predictions"
SILVER_TABLE = f"{CATALOG}.{SCHEMA}.silver_cost_aware_features"

print("Configuration loaded successfully")

---
## 2. Executive Summary Dashboard

### Key Business Metrics at a Glance

This section provides the critical numbers that executives care about:
- Total expected savings
- Investigation efficiency
- Risk coverage

In [ ]:
# SQL Query: Executive Summary Metrics
executive_summary = spark.sql(f"""
    SELECT
        -- Volume Metrics
        COUNT(*) AS total_cases,
        SUM(decision) AS cases_investigated,
        COUNT(*) - SUM(decision) AS cases_not_investigated,
        
        -- Financial Metrics
        ROUND(SUM(expected_savings_if_investigated * decision), 2) AS total_expected_savings,
        ROUND(AVG(CASE WHEN decision = 1 THEN expected_savings_if_investigated END), 2) AS avg_savings_per_investigation,
        ROUND(SUM(investigation_cost * decision), 2) AS total_investigation_cost,
        
        -- Risk Metrics
        ROUND(AVG(risk_probability) * 100, 2) AS avg_risk_probability_pct,
        ROUND(AVG(CASE WHEN decision = 1 THEN risk_probability END) * 100, 2) AS avg_risk_investigated_pct,
        
        -- Efficiency Metrics
        ROUND(
            SUM(expected_savings_if_investigated * decision) / 
            NULLIF(SUM(investigation_cost * decision), 0), 
        2) AS roi_ratio
    FROM {GOLD_TABLE}
""")

display(executive_summary)

In [ ]:
# Convert to pandas for visualization
summary_pdf = executive_summary.toPandas()

# Display formatted metrics
print("="*60)
print("       EXECUTIVE SUMMARY - COST-AWARE AI SYSTEM")
print("="*60)
print(f"\nVOLUME METRICS")
print(f"   Total Cases Analyzed:      {summary_pdf['total_cases'].values[0]:,}")
print(f"   Cases Investigated:        {summary_pdf['cases_investigated'].values[0]:,}")
print(f"   Investigation Rate:        {summary_pdf['cases_investigated'].values[0]/summary_pdf['total_cases'].values[0]*100:.1f}%")

print(f"\nFINANCIAL IMPACT")
print(f"   Total Expected Savings:    ${summary_pdf['total_expected_savings'].values[0]:,.2f}")
print(f"   Avg Savings/Investigation: ${summary_pdf['avg_savings_per_investigation'].values[0]:,.2f}")
print(f"   Total Investigation Cost:  ${summary_pdf['total_investigation_cost'].values[0]:,.2f}")
print(f"   ROI Ratio:                 {summary_pdf['roi_ratio'].values[0]:.2f}x")

print(f"\nRISK COVERAGE")
print(f"   Avg Risk (All Cases):      {summary_pdf['avg_risk_probability_pct'].values[0]:.1f}%")
print(f"   Avg Risk (Investigated):   {summary_pdf['avg_risk_investigated_pct'].values[0]:.1f}%")
print("="*60)

---
## 3. SQL Analytics: Deep Dive Queries

### 3.1 Risk Distribution Analysis

Understanding how risk is distributed across the portfolio helps identify concentration risk and validate model calibration.

In [ ]:
# SQL Query: Risk Distribution by Decile
risk_distribution = spark.sql(f"""
    WITH risk_deciles AS (
        SELECT
            *,
            NTILE(10) OVER (ORDER BY risk_probability) AS risk_decile
        FROM {GOLD_TABLE}
    )
    SELECT
        risk_decile,
        COUNT(*) AS case_count,
        ROUND(MIN(risk_probability) * 100, 2) AS min_risk_pct,
        ROUND(MAX(risk_probability) * 100, 2) AS max_risk_pct,
        ROUND(AVG(risk_probability) * 100, 2) AS avg_risk_pct,
        SUM(decision) AS investigated,
        ROUND(SUM(expected_savings_if_investigated), 2) AS total_potential_savings,
        ROUND(SUM(expected_savings_if_investigated * decision), 2) AS captured_savings
    FROM risk_deciles
    GROUP BY risk_decile
    ORDER BY risk_decile
""")

display(risk_distribution)

In [ ]:
# Visualization: Risk Distribution
risk_pdf = risk_distribution.toPandas()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Chart 1: Case Count by Risk Decile
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, 10))
axes[0].bar(risk_pdf['risk_decile'], risk_pdf['case_count'], color=colors)
axes[0].set_xlabel('Risk Decile (1=Lowest, 10=Highest)', fontsize=11)
axes[0].set_ylabel('Number of Cases', fontsize=11)
axes[0].set_title('Case Distribution by Risk Decile', fontsize=13, fontweight='bold')
axes[0].set_xticks(range(1, 11))

# Chart 2: Potential vs Captured Savings
x = np.arange(len(risk_pdf))
width = 0.35
axes[1].bar(x - width/2, risk_pdf['total_potential_savings'], width, label='Potential Savings', color='lightblue')
axes[1].bar(x + width/2, risk_pdf['captured_savings'], width, label='Captured Savings', color='darkgreen')
axes[1].set_xlabel('Risk Decile', fontsize=11)
axes[1].set_ylabel('Savings ($)', fontsize=11)
axes[1].set_title('Potential vs Captured Savings by Risk Decile', fontsize=13, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(risk_pdf['risk_decile'])
axes[1].legend()

plt.tight_layout()
plt.show()

### 3.2 Investigation Decision Analysis

Compare characteristics of investigated vs. non-investigated cases to validate the optimization logic.

In [ ]:
# SQL Query: Investigated vs Not Investigated Comparison
decision_comparison = spark.sql(f"""
    SELECT
        CASE WHEN decision = 1 THEN 'Investigated' ELSE 'Not Investigated' END AS decision_type,
        COUNT(*) AS case_count,
        ROUND(AVG(risk_probability) * 100, 2) AS avg_risk_pct,
        ROUND(AVG(fraud_loss_if_missed), 2) AS avg_potential_loss,
        ROUND(AVG(investigation_cost), 2) AS avg_investigation_cost,
        ROUND(AVG(expected_savings_if_investigated), 2) AS avg_expected_savings,
        ROUND(SUM(expected_savings_if_investigated), 2) AS total_expected_savings,
        ROUND(MIN(expected_savings_if_investigated), 2) AS min_savings,
        ROUND(MAX(expected_savings_if_investigated), 2) AS max_savings
    FROM {GOLD_TABLE}
    GROUP BY decision
    ORDER BY decision DESC
""")

display(decision_comparison)

In [ ]:
# Visualization: Decision Comparison
decision_pdf = decision_comparison.toPandas()

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

colors = ['#2ecc71', '#e74c3c']

# Chart 1: Case Count
axes[0].pie(decision_pdf['case_count'], labels=decision_pdf['decision_type'], 
            autopct='%1.1f%%', colors=colors, explode=(0.05, 0))
axes[0].set_title('Case Distribution', fontsize=12, fontweight='bold')

# Chart 2: Avg Risk Comparison
bars = axes[1].bar(decision_pdf['decision_type'], decision_pdf['avg_risk_pct'], color=colors)
axes[1].set_ylabel('Average Risk (%)', fontsize=11)
axes[1].set_title('Average Risk by Decision', fontsize=12, fontweight='bold')
for bar, val in zip(bars, decision_pdf['avg_risk_pct']):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, f'{val:.1f}%', 
                 ha='center', fontsize=10, fontweight='bold')

# Chart 3: Avg Expected Savings
bars = axes[2].bar(decision_pdf['decision_type'], decision_pdf['avg_expected_savings'], color=colors)
axes[2].set_ylabel('Avg Expected Savings ($)', fontsize=11)
axes[2].set_title('Avg Expected Savings by Decision', fontsize=12, fontweight='bold')
for bar, val in zip(bars, decision_pdf['avg_expected_savings']):
    axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, f'${val:,.0f}', 
                 ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

### 3.3 High-Value Case Analysis

Identify the highest-impact cases and understand their characteristics.

In [ ]:
# SQL Query: Top 10 Highest Expected Savings Cases
top_cases = spark.sql(f"""
    SELECT
        case_id,
        ROUND(risk_probability * 100, 2) AS risk_pct,
        ROUND(fraud_loss_if_missed, 2) AS potential_loss,
        ROUND(investigation_cost, 2) AS inv_cost,
        ROUND(expected_savings_if_investigated, 2) AS expected_savings,
        priority_rank,
        CASE WHEN decision = 1 THEN 'Yes' ELSE 'No' END AS investigated
    FROM {GOLD_TABLE}
    ORDER BY expected_savings_if_investigated DESC
    LIMIT 10
""")

print("TOP 10 HIGHEST-VALUE CASES")
display(top_cases)

In [ ]:
# SQL Query: Borderline Cases (around capacity threshold)
borderline_cases = spark.sql(f"""
    SELECT
        case_id,
        priority_rank,
        ROUND(risk_probability * 100, 2) AS risk_pct,
        ROUND(expected_savings_if_investigated, 2) AS expected_savings,
        CASE WHEN decision = 1 THEN 'Investigated' ELSE 'Not Investigated' END AS status,
        CASE 
            WHEN priority_rank <= 50 THEN 'Within Capacity'
            ELSE 'Capacity Exceeded'
        END AS capacity_status
    FROM {GOLD_TABLE}
    WHERE priority_rank BETWEEN 45 AND 55
    ORDER BY priority_rank
""")

print("BORDERLINE CASES (Around Capacity Threshold)")
print("These cases show the decision boundary where capacity constraint applies:")
display(borderline_cases)

### 3.4 Financial Loss Prevention Analysis

In [ ]:
# SQL Query: Potential Loss Buckets
loss_buckets = spark.sql(f"""
    SELECT
        CASE
            WHEN fraud_loss_if_missed < 1000 THEN '< $1K'
            WHEN fraud_loss_if_missed < 5000 THEN '$1K - $5K'
            WHEN fraud_loss_if_missed < 10000 THEN '$5K - $10K'
            WHEN fraud_loss_if_missed < 25000 THEN '$10K - $25K'
            ELSE '$25K+'
        END AS loss_bucket,
        COUNT(*) AS total_cases,
        SUM(decision) AS investigated,
        ROUND(SUM(decision) * 100.0 / COUNT(*), 1) AS investigation_rate_pct,
        ROUND(SUM(fraud_loss_if_missed), 2) AS total_potential_loss,
        ROUND(SUM(expected_savings_if_investigated * decision), 2) AS expected_savings_captured
    FROM {GOLD_TABLE}
    GROUP BY 
        CASE
            WHEN fraud_loss_if_missed < 1000 THEN '< $1K'
            WHEN fraud_loss_if_missed < 5000 THEN '$1K - $5K'
            WHEN fraud_loss_if_missed < 10000 THEN '$5K - $10K'
            WHEN fraud_loss_if_missed < 25000 THEN '$10K - $25K'
            ELSE '$25K+'
        END
    ORDER BY 
        MIN(fraud_loss_if_missed)
""")

display(loss_buckets)

In [ ]:
# Visualization: Loss Bucket Analysis
loss_pdf = loss_buckets.toPandas()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Chart 1: Investigation Rate by Loss Bucket
colors = plt.cm.Reds(np.linspace(0.3, 0.8, len(loss_pdf)))
bars = axes[0].bar(loss_pdf['loss_bucket'], loss_pdf['investigation_rate_pct'], color=colors)
axes[0].set_xlabel('Potential Loss Bucket', fontsize=11)
axes[0].set_ylabel('Investigation Rate (%)', fontsize=11)
axes[0].set_title('Investigation Rate by Potential Loss', fontsize=13, fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)
for bar, val in zip(bars, loss_pdf['investigation_rate_pct']):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, f'{val:.1f}%', 
                 ha='center', fontsize=9)

# Chart 2: Total Potential Loss by Bucket
axes[1].bar(loss_pdf['loss_bucket'], loss_pdf['total_potential_loss'], 
            color='lightcoral', label='Total Potential Loss', alpha=0.7)
axes[1].bar(loss_pdf['loss_bucket'], loss_pdf['expected_savings_captured'], 
            color='darkgreen', label='Savings Captured', alpha=0.9)
axes[1].set_xlabel('Potential Loss Bucket', fontsize=11)
axes[1].set_ylabel('Amount ($)', fontsize=11)
axes[1].set_title('Loss Exposure vs Savings Captured', fontsize=13, fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend()

plt.tight_layout()
plt.show()

---
## 4. Model Performance Visualization

### 4.1 ROC Curve and AUC Analysis

In [ ]:
# Get predictions and labels for ROC curve
roc_data = spark.sql(f"""
    SELECT 
        r.risk_probability,
        s.label
    FROM {RISK_TABLE} r
    JOIN {SILVER_TABLE} s ON r.case_id = s.case_id
""").toPandas()

# Calculate ROC curve points
from sklearn.metrics import roc_curve, auc, precision_recall_curve

fpr, tpr, thresholds = roc_curve(roc_data['label'], roc_data['risk_probability'])
roc_auc = auc(fpr, tpr)

# Plot ROC Curve
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC Curve
axes[0].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC Curve (AUC = {roc_auc:.3f})')
axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
axes[0].fill_between(fpr, tpr, alpha=0.3, color='orange')
axes[0].set_xlim([0.0, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].set_xlabel('False Positive Rate', fontsize=11)
axes[0].set_ylabel('True Positive Rate', fontsize=11)
axes[0].set_title('ROC Curve - Risk Prediction Model', fontsize=13, fontweight='bold')
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(roc_data['label'], roc_data['risk_probability'])
pr_auc = auc(recall, precision)

axes[1].plot(recall, precision, color='green', lw=2, label=f'PR Curve (AUC = {pr_auc:.3f})')
axes[1].fill_between(recall, precision, alpha=0.3, color='green')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('Recall', fontsize=11)
axes[1].set_ylabel('Precision', fontsize=11)
axes[1].set_title('Precision-Recall Curve', fontsize=13, fontweight='bold')
axes[1].legend(loc='lower left')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nModel Performance Summary:")
print(f"   ROC-AUC: {roc_auc:.4f}")
print(f"   PR-AUC:  {pr_auc:.4f}")

### 4.2 Risk Probability Distribution

In [ ]:
# Visualization: Risk Score Distribution by Actual Label
fig, ax = plt.subplots(figsize=(10, 5))

fraud_scores = roc_data[roc_data['label'] == 1]['risk_probability']
non_fraud_scores = roc_data[roc_data['label'] == 0]['risk_probability']

ax.hist(non_fraud_scores, bins=30, alpha=0.6, label='Non-Fraud', color='green', density=True)
ax.hist(fraud_scores, bins=30, alpha=0.6, label='Fraud', color='red', density=True)

ax.set_xlabel('Risk Probability', fontsize=11)
ax.set_ylabel('Density', fontsize=11)
ax.set_title('Risk Score Distribution: Fraud vs Non-Fraud', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 5. Cost-Benefit Analysis

### 5.1 Cumulative Savings Curve

This shows how much savings we capture as we increase investigation capacity.

In [ ]:
# SQL Query: Cumulative Savings by Priority
cumulative_savings = spark.sql(f"""
    SELECT
        priority_rank,
        expected_savings_if_investigated,
        SUM(expected_savings_if_investigated) OVER (
            ORDER BY priority_rank
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS cumulative_savings,
        SUM(investigation_cost) OVER (
            ORDER BY priority_rank
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS cumulative_cost
    FROM {GOLD_TABLE}
    ORDER BY priority_rank
""").toPandas()

# Plot cumulative savings curve
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Chart 1: Cumulative Savings
axes[0].plot(cumulative_savings['priority_rank'], cumulative_savings['cumulative_savings'], 
             color='green', lw=2, label='Cumulative Expected Savings')
axes[0].axvline(x=50, color='red', linestyle='--', lw=2, label='Capacity Limit (50)')
axes[0].fill_between(cumulative_savings['priority_rank'][:50], 
                     cumulative_savings['cumulative_savings'][:50], 
                     alpha=0.3, color='green')
axes[0].set_xlabel('Number of Cases Investigated (Ranked by Priority)', fontsize=11)
axes[0].set_ylabel('Cumulative Expected Savings ($)', fontsize=11)
axes[0].set_title('Cumulative Savings vs Investigation Count', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Chart 2: Marginal Savings (diminishing returns)
axes[1].bar(cumulative_savings['priority_rank'][:100], 
            cumulative_savings['expected_savings_if_investigated'][:100],
            color='steelblue', alpha=0.7)
axes[1].axvline(x=50, color='red', linestyle='--', lw=2, label='Capacity Limit')
axes[1].set_xlabel('Priority Rank', fontsize=11)
axes[1].set_ylabel('Expected Savings per Case ($)', fontsize=11)
axes[1].set_title('Marginal Savings by Priority (Diminishing Returns)', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 5.2 Capacity Sensitivity Analysis

How do results change with different capacity constraints?

In [ ]:
# Sensitivity Analysis: Different Capacity Levels
capacity_levels = [25, 50, 75, 100, 150, 200]
sensitivity_results = []

for cap in capacity_levels:
    result = spark.sql(f"""
        SELECT
            {cap} AS capacity,
            SUM(CASE WHEN priority_rank <= {cap} THEN expected_savings_if_investigated ELSE 0 END) AS total_savings,
            SUM(CASE WHEN priority_rank <= {cap} THEN investigation_cost ELSE 0 END) AS total_cost,
            SUM(CASE WHEN priority_rank <= {cap} THEN 1 ELSE 0 END) AS cases_investigated
        FROM {GOLD_TABLE}
    """).toPandas()
    sensitivity_results.append(result)

sensitivity_df = pd.concat(sensitivity_results, ignore_index=True)
sensitivity_df['roi'] = sensitivity_df['total_savings'] / sensitivity_df['total_cost']
sensitivity_df['savings_per_case'] = sensitivity_df['total_savings'] / sensitivity_df['cases_investigated']

display(sensitivity_df)

In [ ]:
# Visualization: Capacity Sensitivity
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Chart 1: Total Savings by Capacity
axes[0].plot(sensitivity_df['capacity'], sensitivity_df['total_savings'], 
             marker='o', color='green', lw=2, markersize=8)
axes[0].axvline(x=50, color='red', linestyle='--', alpha=0.7, label='Current Capacity')
axes[0].set_xlabel('Investigation Capacity', fontsize=11)
axes[0].set_ylabel('Total Expected Savings ($)', fontsize=11)
axes[0].set_title('Total Savings vs Capacity', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Chart 2: ROI by Capacity
axes[1].plot(sensitivity_df['capacity'], sensitivity_df['roi'], 
             marker='s', color='blue', lw=2, markersize=8)
axes[1].axvline(x=50, color='red', linestyle='--', alpha=0.7, label='Current Capacity')
axes[1].set_xlabel('Investigation Capacity', fontsize=11)
axes[1].set_ylabel('ROI (Savings / Cost)', fontsize=11)
axes[1].set_title('ROI vs Capacity', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Chart 3: Savings per Case (diminishing returns)
axes[2].plot(sensitivity_df['capacity'], sensitivity_df['savings_per_case'], 
             marker='^', color='purple', lw=2, markersize=8)
axes[2].axvline(x=50, color='red', linestyle='--', alpha=0.7, label='Current Capacity')
axes[2].set_xlabel('Investigation Capacity', fontsize=11)
axes[2].set_ylabel('Avg Savings per Case ($)', fontsize=11)
axes[2].set_title('Diminishing Returns Analysis', fontsize=12, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nINSIGHT: Increasing capacity from 50 to 75 would capture additional savings,")
print("   but with lower ROI per case due to diminishing returns.")

---
## 6. Baseline Comparison: Why Cost-Aware Matters

Compare our cost-aware approach vs. naive strategies.

In [ ]:
# SQL Query: Baseline Comparison
baseline_comparison = spark.sql(f"""
    WITH ranked_data AS (
        SELECT
            case_id,
            risk_probability,
            expected_savings_if_investigated,
            investigation_cost,
            fraud_loss_if_missed,
            -- Our cost-aware ranking
            ROW_NUMBER() OVER (ORDER BY expected_savings_if_investigated DESC) AS cost_aware_rank,
            -- Baseline 1: Highest risk first
            ROW_NUMBER() OVER (ORDER BY risk_probability DESC) AS risk_first_rank,
            -- Baseline 2: Highest loss first
            ROW_NUMBER() OVER (ORDER BY fraud_loss_if_missed DESC) AS loss_first_rank,
            -- Baseline 3: Random (using case_id hash as proxy)
            ROW_NUMBER() OVER (ORDER BY HASH(case_id)) AS random_rank
        FROM {GOLD_TABLE}
    )
    SELECT
        'Cost-Aware (Ours)' AS strategy,
        ROUND(SUM(CASE WHEN cost_aware_rank <= 50 THEN expected_savings_if_investigated ELSE 0 END), 2) AS total_savings,
        ROUND(AVG(CASE WHEN cost_aware_rank <= 50 THEN expected_savings_if_investigated END), 2) AS avg_savings
    FROM ranked_data
    
    UNION ALL
    
    SELECT
        'Risk-First (Baseline)' AS strategy,
        ROUND(SUM(CASE WHEN risk_first_rank <= 50 THEN expected_savings_if_investigated ELSE 0 END), 2),
        ROUND(AVG(CASE WHEN risk_first_rank <= 50 THEN expected_savings_if_investigated END), 2)
    FROM ranked_data
    
    UNION ALL
    
    SELECT
        'Loss-First (Baseline)' AS strategy,
        ROUND(SUM(CASE WHEN loss_first_rank <= 50 THEN expected_savings_if_investigated ELSE 0 END), 2),
        ROUND(AVG(CASE WHEN loss_first_rank <= 50 THEN expected_savings_if_investigated END), 2)
    FROM ranked_data
    
    UNION ALL
    
    SELECT
        'Random (Baseline)' AS strategy,
        ROUND(SUM(CASE WHEN random_rank <= 50 THEN expected_savings_if_investigated ELSE 0 END), 2),
        ROUND(AVG(CASE WHEN random_rank <= 50 THEN expected_savings_if_investigated END), 2)
    FROM ranked_data
""")

display(baseline_comparison)

In [ ]:
# Visualization: Strategy Comparison
baseline_pdf = baseline_comparison.toPandas()

fig, ax = plt.subplots(figsize=(10, 6))

colors = ['#2ecc71', '#3498db', '#9b59b6', '#95a5a6']
bars = ax.barh(baseline_pdf['strategy'], baseline_pdf['total_savings'], color=colors)

# Add value labels
for bar, val in zip(bars, baseline_pdf['total_savings']):
    ax.text(val + 1000, bar.get_y() + bar.get_height()/2, 
            f'${val:,.0f}', va='center', fontsize=11, fontweight='bold')

ax.set_xlabel('Total Expected Savings ($)', fontsize=12)
ax.set_title('Strategy Comparison: Cost-Aware vs Baselines\n(50 Investigations)', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

# Calculate improvement
our_savings = baseline_pdf[baseline_pdf['strategy'] == 'Cost-Aware (Ours)']['total_savings'].values[0]
risk_savings = baseline_pdf[baseline_pdf['strategy'] == 'Risk-First (Baseline)']['total_savings'].values[0]
improvement = (our_savings - risk_savings) / risk_savings * 100

print(f"\nKEY FINDING: Cost-Aware approach captures {improvement:.1f}% more savings")
print(f"   than the traditional risk-first approach!")

---
## 7. Final Executive Dashboard Summary

In [ ]:
# Final Summary Dashboard
print("\n" + "="*70)
print("           COST-AWARE AI DECISION SYSTEM - FINAL REPORT")
print("="*70)

print("\nSYSTEM OVERVIEW")
print("-"*70)
print("   Objective: Optimize fraud investigation decisions under capacity")
print("              constraints by maximizing expected financial savings.")
print("   Approach:  ML-driven risk prediction + Cost-aware optimization")

print("\nKEY PERFORMANCE METRICS")
print("-"*70)
print(f"   * Model ROC-AUC:              {roc_auc:.4f}")
print(f"   * Total Cases Analyzed:       {summary_pdf['total_cases'].values[0]:,}")
print(f"   * Cases Investigated:         {summary_pdf['cases_investigated'].values[0]:,}")
print(f"   * Total Expected Savings:     ${summary_pdf['total_expected_savings'].values[0]:,.2f}")
print(f"   * ROI Ratio:                  {summary_pdf['roi_ratio'].values[0]:.2f}x")

print("\nKEY INSIGHTS")
print("-"*70)
print(f"   * Cost-aware approach outperforms risk-first by {improvement:.1f}%")
print("   * Diminishing returns evident beyond capacity threshold")
print("   * High-value cases successfully prioritized")

print("\nBUSINESS RECOMMENDATIONS")
print("-"*70)
print("   1. Implement cost-aware prioritization for daily investigations")
print("   2. Consider capacity expansion to 75 for additional savings")
print("   3. Monitor model performance for drift detection")
print("   4. Review borderline cases for manual override criteria")

print("\n" + "="*70)
print("         System demonstrates measurable business value")
print("="*70)

---
## Notebook Summary

This dashboard provides:

1. **Executive Summary** - Key metrics at a glance
2. **SQL Analytics** - Deep-dive queries for various analyses
3. **Risk Distribution** - Understanding portfolio risk concentration
4. **Model Performance** - ROC curves and calibration analysis
5. **Cost-Benefit Analysis** - Cumulative savings and marginal returns
6. **Sensitivity Analysis** - Impact of capacity changes
7. **Baseline Comparison** - Proof of cost-aware superiority

**Key Finding**: The cost-aware approach significantly outperforms traditional methods by optimizing for business value rather than prediction accuracy alone.